In [6]:
# Import for data manipulation
import pandas as pd 
import sklearn as sk
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import statsmodels.api as sm
import ast


from scipy.stats import f_oneway

from sklearn.impute import SimpleImputer


# Import package for sentiment analysis
import nltk
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords

In [7]:
# Download for sentiment analysis
nltk.download('punkt')
nltk.download('stopwords')

[nltk_data] Downloading package punkt to /home/ealopez/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     /home/ealopez/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [32]:
#Data Cleaning

In [38]:
df_habits = pd.read_csv('grocery_habits.csv')
df_habits = df_habits.drop(columns = 'Timestamp')

In [39]:
df_habits = df_habits.rename(columns={"Does most of your food waste come from prepared foods or from groceries themselves? Fun fact: Per 2/4 rule by the FDA, if you put the leftovers in fridge within 2 hours it’s good for 4 days." : "Prepared or Grocery Waste"})
df_habits = df_habits.rename(columns={"If you could come up with a solution to these problems, what would you want?" : "Solution"})
df_habits = df_habits.rename(columns={"Do you already use apps to track groceries' expiration dates?" : "App"})

In [33]:
#Sentiment Analysis